In [1]:
# Run the following commands to get your hostname and pluggable database service name
import os
print('Hostname: ',os.uname()[1])
stream = os.popen('lsnrctl status | grep mlpdb1')
print(stream.read())

Hostname:  omlvm.livelabs.oraclevcn.com
Service "mlpdb1.livelabs.oraclevcn.com" has 1 instance(s).



## **Task 1:** Prepare and explore data 

In [2]:

import oml
import pandas as pd
oml.connect(user="oml_user", password="MLlearnPTS#21_", 
            host="oml001", port=1521, 
            service_name="mlpdb1.sub07141037280.rehevcn.oraclevcn.com",
            automl=True)
oml.isconnected()

DatabaseError: ORA-12545: Connect failed because target host or object does not exist

In [ ]:
# Open a cursor and verify database version.
cr = oml.cursor()
conn = cr.connection
print("Database version:", conn.version)

In [ ]:
# Retrieve user tables from the database schema.
cr.execute("select table_name, num_rows from user_tables order by 1").fetchall()
# When you execute a query using an oml.cursor, the result is a Python list.

In [ ]:
# It is always a good practice to close the cursor once your transaction is completed.
cr.close()

In [ ]:
# You can use external data sources for educational purposes.
from pydataset import data
data()

In [ ]:
# Filter properties and methods list using pattern '*Ins*' and wildcard.
import fnmatch
load_list = fnmatch.filter(data().dataset_id, '*Ins*')
print(load_list)

In [ ]:
# Load Insurance dataset as a Pandas data frame.
claims_df = data('Insurance')
claims_df

In [ ]:
# Create a table in the database schema and return an OML data frame (oml.DataFrame) object 
# that is a proxy for the table.
try:
   oml.drop('CUST_INSUR_CLAIMS') 
except: 
   pass
oml_claims = oml.create(claims_df, table = 'CUST_INSUR_CLAIMS')
# Notice you try to drop CUST_INSUR_CLAIMS table before creating it, just to make sure it 
# doesn't exist already and you get an error.

In [ ]:
# Refresh OML_USER tables in SQL Developer, and verify the data in new table CUST_INSUR_CLAIMS.
cr = oml.cursor()
cr.execute("select * from CUST_INSUR_CLAIMS").fetchall()

In [ ]:
# Check the type of claims_df object.
type(claims_df)

In [ ]:
# Check the type of oml_claims object.
type(oml_claims)
# It is recommended to verify the type every time you create a new Python object. It is also
# recommended to return a the list of attributes of the object using dir(object_name) function.

In [ ]:
# Check the columns of oml_claims data frame, using OML data frame columns attribute.
oml_claims.columns

In [ ]:
# Display the values in oml_claims data frame. As it has 15k records, the list is truncated.
oml_claims

In [ ]:
# Retrieve OML_USER tables column details and close the cursor when finished.
cr = oml.cursor()
col_list = cr.execute("select table_name, column_name, data_type, data_length \
                     from user_tab_cols \
                     where table_name like 'CUST_%' \
                     order by 1,2").fetchall()
cr.close()
col_list

In [ ]:
# Check the type of col_list object.
type(col_list)

In [ ]:
# Check the type of one element in col_list object.
type(('CUST_INSUR_LTV', 'T_AMOUNT_AUTOM_PAYMENTS', 'NUMBER', 22))

In [ ]:
# Create a Pandas data frame using records from col_list object.
col_df = pd.DataFrame(col_list, columns = ['TABLE_NAME', 'COLUMN_NAME', 
                                           'DATA_TYPE', 'DATA_LENGTH'])
col_df

In [ ]:
# Create a new OML data frame object that contains two columns from 
# oml_claims object, filtered by Age.
oml_claims_24 = oml_claims[oml_claims["Age"] == '<25',
                           ["Holders", "Claims"]]
oml_claims_24

In [ ]:
# Return the sum of the values, representing the total number of 
# claims for drivers under 25.
oml_claims_24.sum()

In [ ]:
# Calculate risk factor for drivers under 25, as number of claims 
# divided by the number of holders.
oml_claims_24.sum().Claims/oml_claims_24.sum().Holders

In [ ]:
# Generate a Pandas data frame from a database table. CUST_INSUR_LTV is the table 
# you will use during the entire workshop, containing insurance customers data.
cust_df = oml.sync(table = "CUST_INSUR_LTV").pull()
type(cust_df)

In [ ]:
# Display the first records in cust_df Pandas data frame, by default 10.
cust_df.head()

In [ ]:
# Generate an OML data frame from your database table.
oml_cust = oml.sync(table = 'CUST_INSUR_LTV')
type(oml_cust)

In [ ]:
# Display the first records in oml_cust OML data frame, by default 10.
oml_cust.head()

In [ ]:
# Life-time value (LTV) is stored in LTV column, and predefined classes based on this 
# value are stored in LTV_BIN column. These classes have been defined by a human operator. 
# Keep this in mind when you are testing clustering algorithms for market segmentation, the 
# machine doesn't have to follow same limits as the ones specified by the business user.
classes = oml.sync(query = 'select LTV_BIN, min(LTV), max(LTV) \
                            from CUST_INSUR_LTV group by LTV_BIN order by 2')
classes

**Note** : As you can see, our data is pre-labeled. However, you will drop the labels you don't need during training phase, depending on the algorithm. You will drop all labels during testing phase, and verify predicted labels with the pre-label information to check your model accuracy.


In [ ]:
# Plot the distribution of customers by LTV. The most common approach to 
# visualizing a distribution is the histogram.
import matplotlib.pyplot as plt
oml.graphics.hist(oml_cust['LTV'], 'auto', color='orange',
                  linestyle='solid', edgecolor='white') 
plt.title('Customers distribution by LTV')
plt.ylabel('Customers') 
plt.xlabel('LTV')

## **Task 2:** Perform customer classification using a Neural Network

Neural Network (NN) algorithms can be used for regression and classification machine learning functions. There are two types of classification:

- Binary classification refers to predicting one of two classes for each customer case. E.g. if a customer will buy insurance or not: Yes, No.
- Multi-label classification refers to predicting one of more than two classes for each customer. E.g. what size is a customer: Low, Medium, Large, Very Large.

An artificial neural network is composed of a large number of interconnected neurons which exchange messages between each other to solve specific problems. They learn by examples and tune the weights of the connections among the neurons during the learning process.

> **Note** : For more information, visit [Neural Network](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/neural-network.html#GUID-C45971D9-A874-4546-A0EC-1FF25B229E2B) documentation.

At this step, you will build, test, and tune a neural network that can classify your customers in four `LTV_BIN` classes (*LOW*, *MEDIUM*, *HIGH*, and *VERY HIGH*). You will test both types of classification.


In [ ]:
# Generate an OML data frame from your database table.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust = oml_cust.drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set into training and test data.
ltv_dat = oml_cust.split() 
[split.shape for split in ltv_dat]

In [ ]:
# Create training data and test data.
train_x = ltv_dat[0].drop('LTV_BIN') 
train_y = ltv_dat[0]['LTV_BIN']
test_ltv = ltv_dat[1]

In [ ]:
# Create a NN model object.
nn_mod = oml.nn(nnet_hidden_layers = 1,
                nnet_activations= "'NNET_ACTIVATIONS_LOG_SIG'", 
                NNET_NODES_PER_LAYER= '30')

**Note** : For more information, visit [Algorithm Settings: Neural Network](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-7793F608-2719-45EA-87F9-6F246BA800D4) documentation.
    
**Note** : To understand this model, visit the [Neural Network](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/neural-network.html#GUID-27FE0680-91A9-4F44-B69C-134E3D3BEEC8) page in OML user guide.


In [ ]:
# Fit the NN model according to the training data and parameter settings.
nn_mod = nn_mod.fit(train_x, train_y)

In [ ]:
# Show details of the model. Did your model converge? If yes, after how many iterations?
nn_mod

In [ ]:
# Use the model to make predictions on test data.
nn_mod.predict(test_ltv.drop('LTV_BIN'),
               supplemental_cols = test_ltv[:, ['CUST_ID','LAST',
                                                'FIRST','LTV_BIN']]).head(25)

In [ ]:
# Return the prediction probability.
nn_mod.predict(test_ltv.drop('LTV_BIN'),
               supplemental_cols = test_ltv[:, ['CUST_ID','LAST',
                                                'FIRST','LTV_BIN']], 
               proba = True).head(25)

In [ ]:
# Return mean accuracy for classification. How accurate is your model?
nn_mod.score(test_ltv.drop('LTV_BIN'), test_ltv[:, ['LTV_BIN']])

In [ ]:
# Change the setting parameter and refit the model. After how 
# many iterations did this model converge?
new_setting = {'NNET_NODES_PER_LAYER': '50'} 
nn_mod.set_params(**new_setting).fit(train_x, train_y)

In [ ]:
# Return new mean accuracy for classification. How is the 
# new accuracy compared to the previous one?
nn_mod.score(test_ltv.drop('LTV_BIN'), test_ltv[:, ['LTV_BIN']])

In [ ]:
# Export the NN model as a serialized model to a new table 
# named NN_MODEL in the database.
try:
   oml.drop('NN_MODEL') 
except: 
   pass
nn_export = nn_mod.export_sermodel(table='NN_MODEL')
type(nn_export)

In [ ]:
# Show the first 10 characters of the BLOB content from the 
# serialized model export.
nn_export.pull()[0][1:10]

In [ ]:
# You can use a NN model for Binary Classification. In this case you 
# can predict if customer will buy or not the insurance, column BUY_INSURANCE 
# in the data set. Re-generate an OML data frame.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust.head()

In [ ]:
# Split the data set and create training and test data.
ltv_dat = oml_cust.split() 
train_x = ltv_dat[0].drop('BUY_INSURANCE') 
train_y = ltv_dat[0]['BUY_INSURANCE']
test_ltv = ltv_dat[1]

In [ ]:
# Create a NN model object, with new settings and refit the model.
setting = {'nnet_hidden_layers': 1, 
           'nnet_activations': 'NNET_ACTIVATIONS_LOG_SIG', 
           'NNET_NODES_PER_LAYER': '30'} 
nn_mod.set_params(**setting).fit(train_x, train_y)

In [ ]:
# Use the model to make predictions on test data. This time 
# it predicts if customers will buy on not insurance.
nn_mod.predict(test_ltv.drop('BUY_INSURANCE'),
               supplemental_cols = test_ltv[:, ['CUST_ID','LAST',
                                                'FIRST','BUY_INSURANCE']]).head(25)

In [ ]:
# Return mean accuracy for the binary classification. How 
# accurate is your model? Can you improve this accuracy?
nn_mod.score(test_ltv.drop('BUY_INSURANCE'), test_ltv[:, ['BUY_INSURANCE']])

## **Task 3:** Classification model using Decision Tree algorithm

Decision Tree (DT) is a supervised machine learning algorithm used for classifying data. In some applications of Oracle Machine Learning, the ability to explain the reason for a decision can be crucial. Decision trees generate rules. A rule is a conditional statement that can be understood by humans and used within a database to identify a set of records.

> **Note** : For more information on this algorithm, visit [Decision Tree](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/decision-tree.html#GUID-14DE1A88-220F-44F0-9AC8-77CA844D4A63) documentation.

In this example, you will not only classify your customers in four `LTV_BIN` classes (*LOW*, *MEDIUM*, *HIGH*, and *VERY HIGH*), but you will also retrieve the rules (conditions) that are behind the customers classification.


In [ ]:
# Create the OML data frame for this step. Drop LTV column as you will use 
# only the LTV_BIN classes for the predicted value.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust = oml_cust.drop('LTV')
oml_cust.head()

In [ ]:
# Verify the shape of the OML data frame.
oml_cust.shape

In [ ]:
# Split the data set into training and test data. Use 80% for train and 20% for test ratio.
ltv_dat = oml_cust.split(ratio=(.8, .2)) 
[split.shape for split in ltv_dat]

In [ ]:
# Create training data and test data.
train_x = ltv_dat[0].drop('LTV_BIN') 
train_y = ltv_dat[0]['LTV_BIN']
test_ltv = ltv_dat[1]

In [ ]:
# Verify the four LTV_BIN classes (LOW, MEDIUM, HIGH, and VERY HIGH) in your dataset.
cr = oml.cursor()
cr.execute("select unique LTV_BIN from CUST_INSUR_LTV order by 1").fetchall()

In [ ]:
# Close the cursor.
cr.close()

In [ ]:
# Create a cost matrix table in the database. A cost matrix is a mechanism 
# for influencing the decision making of a model. In this case, the cost 
# matrix will cause the model to minimize costly misclassifications.
try:
   oml.drop('LTV_COST_MATRIX') 
except: 
   pass
cost_matrix = [['LOW', 'LOW', 0],
               ['LOW', 'MEDIUM', 0.3], 
               ['LOW', 'HIGH', 0.3], 
               ['LOW', 'VERY HIGH', 0.4], 
               ['MEDIUM', 'LOW', 0.4], 
               ['MEDIUM', 'MEDIUM', 0], 
               ['MEDIUM', 'HIGH', 0.3], 
               ['MEDIUM', 'VERY HIGH', 0.3], 
               ['HIGH', 'LOW', 0.5], 
               ['HIGH', 'MEDIUM', 0.3], 
               ['HIGH', 'HIGH', 0], 
               ['HIGH', 'VERY HIGH', 0.2], 
               ['VERY HIGH', 'LOW', 0.6], 
               ['VERY HIGH', 'MEDIUM', 0.3], 
               ['VERY HIGH', 'HIGH', 0.1], 
               ['VERY HIGH', 'VERY HIGH', 0]]
cost_matrix = oml.create( pd.DataFrame(cost_matrix,
                          columns = ['ACTUAL_TARGET_VALUE', 
                                     'PREDICTED_TARGET_VALUE', 
                                     'COST']),
                          table = 'LTV_COST_MATRIX')

**Note** : For more information, visit [Cost Matrix Table](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-CF6EB584-8FE9-44F5-BAC0-0751DC094CCE__CACBEFFJ) documentation.


In [ ]:
# Specify algorithm settings.
setting = {'TREE_TERM_MAX_DEPTH':'8'}

**Note** : For more information, visit [Algorithm Settings: Decision Tree](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-03435110-D723-42FD-B4EA-39C86A039566) documentation.


In [ ]:
# Create a DT model object.
dt_mod = oml.dt(**setting)

**Note** : For more information, visit [Algorithm Settings: Decision Tree](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-03435110-D723-42FD-B4EA-39C86A039566) documentation.


In [ ]:
# Fit the DT model according to the training data and parameter settings.
dt_mod.fit(train_x, train_y, cost_matrix = cost_matrix)

In [ ]:
# Use the model to make predictions on the test data.
predict_dat = dt_mod.predict(test_ltv.drop('LTV_BIN'),
                             supplemental_cols = test_ltv[:, ['CUST_ID','LAST','FIRST','LTV_BIN']])
predict_dat.sort_values('CUST_ID')

In [ ]:
# Confusion matrix is a technique for summarizing the performance of a 
# classification model. This matrix shows the number of customers pre-labeled with 
# LTV_BIN value and predicted as PREDICTION value in count(CUST_ID)_(PREDICTION) columns.
predict_dat[['LTV_BIN','PREDICTION',
             'CUST_ID']].pivot_table('LTV_BIN', 'PREDICTION', 
                                         aggfunc = oml.DataFrame.count)

In [ ]:
# Show only customers with wrong predictions.
test_predict = test_ltv[['CUST_ID','LAST','FIRST',
                         'LTV_BIN']].merge(other = predict_dat[['CUST_ID',
                                                                'PREDICTION']], 
                                                   on="CUST_ID")
test_predict[test_predict['LTV_BIN_l'] != 
             test_predict['PREDICTION_r']].sort_values('CUST_ID')

In [ ]:
# Make predictions and return the probability. DT model returns 
# prediction probability if proba parameter is True.
predict_dat = dt_mod.predict(test_ltv.drop('LTV_BIN'),
                             supplemental_cols = test_ltv[:, ['CUST_ID','LAST',
                                                              'FIRST','LTV_BIN']],
                             proba = True)
predict_dat.sort_values('CUST_ID')

In [ ]:
# Show only customers with wrong predictions and the probability.
test_predict = test_ltv[['CUST_ID','LAST',
                         'FIRST','LTV_BIN']].merge(other = predict_dat[['CUST_ID','PREDICTION',
                                                                        'PROBABILITY']], 
                                                   on="CUST_ID")
test_predict[test_predict['LTV_BIN_l'] != test_predict['PREDICTION_r']].sort_values('PROBABILITY_r')

In [ ]:
# Calculate the DT model score. DT model score attribute returns the mean accuracy.
dt_mod.score(test_ltv.drop('LTV_BIN'), test_ltv[:, ['LTV_BIN']])

In [ ]:
# Reset TREE_TERM_MAX_DEPTH and refit model. This setting represents 
# the maximum number of nodes between the root and any leaf node, 
# including the leaf node. More nodes means longer time to train the DT model. 
# You may grab a coffee until this step is completed.
dt_mod.set_params(TREE_TERM_MAX_DEPTH = '9').fit(train_x, train_y, cost_matrix = cost_matrix)

In [ ]:
# Re-calculate the model score. How much did it improve? You can decide if 
# it is worth the time spent.
dt_mod.score(test_ltv.drop('LTV_BIN'), test_ltv[:, ['LTV_BIN']])

In [ ]:
# The good thing is that once you have a great model, it can be exported 
# and reused, even to other databases, e.g. Oracle Autonomous Database. 
# Export serialized model to a table.
dt_export = dt_mod.export_sermodel(table='dt_sermod')

In [ ]:
# Your DT model is exported as a binary object.
type(dt_export)

In [ ]:
# Show the first 100 characters of the BLOB content from the model export.
dt_export.pull()[0][1:100]

## **Task 4:** Predict LTV values using Generalized Linear Model regression

Generalized Linear Model (GLM) is a statistical technique used for linear modeling. Oracle Machine Learning supports GLM for regression and binary classification. Regression is a supervised learning technique used to a predict continuous response target value based on independent predictors.

> **Note** : For more information on this algorithm, visit [Generalized Linear Model](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/generalized-linear-model.html#GUID-5E59530F-EBD9-414E-8C8B-63F8079772CE) documentation.

In your case, the independent predictors are customer attributes, and the response target value is the customer life-time value.


In [ ]:
# Create an OML data frame proxy object in Python that represents your Oracle Database data set.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust = oml_cust.drop('LTV_BIN')
oml_cust.head()

In [ ]:
# Split the data set into training and test data, using default ratio.
ltv_dat = oml_cust.split() 
[split.shape for split in ltv_dat]

In [ ]:
# Create training data and test data.
train_x = ltv_dat[0].drop('LTV') 
train_y = ltv_dat[0]['LTV']
test_ltv = ltv_dat[1]
[frame.shape for frame in (train_x, train_y, test_ltv)]

In [ ]:
# Specify settings.
setting = {'GLMS_SOLVER': 'dbms_data_mining.GLMS_SOLVER_SGD'}

**Note** : For more information, visit [Algorithm Settings: Generalized Linear Models](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-4E3665B9-B1C2-4F6B-AB69-A7F353C70F5C) documentation.


In [ ]:
# Create a GLM model object.
glm_mod = oml.glm('REGRESSION', **setting)

**Note** : To understand this model, visit the [Generalized Linear Models](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/general-linearized-model.html#GUID-4464A453-60F0-4751-B231-91BC5708D1F8) page in OML user guide.


In [ ]:
# Fit the GLM model according to the training data and parameter settings. 
# The name of a column that contains unique case identifiers is used for case_id parameter.
glm_mod = glm_mod.fit(train_x, train_y, case_id = 'CUST_ID')

In [ ]:
# Show the model details.
glm_mod

In [ ]:
# Check the value of converged attribute.
glm_mod.converged

Convergence means the optimal solution has been reached and the iterations of the optimization has come to an end. Can you make predictions if converged attribute is False?

In [ ]:
# Specify new settings.
setting = {'GLMS_SOLVER': 'dbms_data_mining.GLMS_SOLVER_CHOL'}

In [ ]:
# Recreate a GLM model object with new settings.
glm_mod = oml.glm('REGRESSION', **setting)

In [ ]:
# Refit the GLM model according to the training data and parameter settings.
glm_mod = glm_mod.fit(train_x, train_y, case_id = 'CUST_ID')

In [ ]:
# Recheck the value of converged attribute.
glm_mod.converged

In [ ]:
# Use the model to make predictions on the test data. How are 
# the PREDICTION values compared to the exact LTV values?
glm_mod.predict(test_ltv.drop('LTV'),
                supplemental_cols = test_ltv[:, ['CUST_ID','LAST','FIRST','LTV']])

In [ ]:
# Return the prediction probability.
glm_mod.predict(test_ltv.drop('LTV'),
                supplemental_cols = test_ltv[:, ['CUST_ID','LAST','FIRST','LTV']], 
                proba = True)

In [ ]:
# How far are predicted values from the pre-labeled LTV values you have 
# in the dataset? Create an OML data frame to investigate.
predictions = glm_mod.predict(test_ltv.drop('LTV'),
                supplemental_cols = test_ltv[:, ['LTV']])
predictions

In [ ]:
# Calculate the differences between pre-labeled LTV and predicted LTV values.
diff = (predictions['LTV']-predictions['PREDICTION'])
diff

In [ ]:
# Concatenate OML data frame with resulted differences.
predictions.rename({'LTV':'LABEL_LTV'})
ltv_diff = predictions.concat(diff).rename({'LTV':'LTV_DIFFERENCE'})
ltv_diff

In [ ]:
# Plot the prediction error values.
import matplotlib.pyplot as plt
plt.stem(ltv_diff.pull()[['LABEL_LTV']], 
         ltv_diff.pull()[['LTV_DIFFERENCE']], 'b.')
plt.xlabel('Label LTV value')
plt.ylabel('LTV difference')
plt.title('Prediction error from prelabeled LTV')
plt.show()

## **Task 5:** Discover natural groupings with K-Means clustering

Oracle Machine Learning supports enhanced k-Means (KM) clustering algorithm. The KM algorithm is a distance-based clustering algorithm that partitions the data into a specified number of clusters. It relies on a distance function to measure the similarity between cases. Cases are assigned to the nearest cluster according to the distance function used.

> **Note** : For more information on this algorithm, visit [k-Means](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/k-means.html#GUID-AA5D4D4E-936F-474A-8919-5E7FF5EE69B1) documentation.

You will group customers in four clusters. KM algorithm is more appropriate for data sets with a low number of attributes. For this reason, you will reduce the training and test data to just one dimension, customer life-time value.


In [ ]:
# Create training and test data. Use only CUST_ID column that contains unique 
# case identifiers, and LTV column that contains customer life-time values.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust_one = oml_cust[['CUST_ID','LTV']]
oml_cust_one.head()

In [ ]:
# Left merge the oml_cust full data set to verify predefined classes stored in 
# LTV_BIN column to the reduced data set called oml_cust_one.
oml_cust_one.merge(other=oml_cust[['CUST_ID', 'LTV_BIN']], on="CUST_ID")

In [ ]:
# Split the data set and create training and test data.
ltv_dat = oml_cust_one.split() 
train_ltv = ltv_dat[0]
test_ltv = ltv_dat[1]
[frame.shape for frame in (train_ltv, test_ltv)]

In [ ]:
# Specify settings.
setting = {'kmns_iterations': 20}

**Note** : For more information, visit [Algorithm Settings: k-Means](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-7010593E-C323-4DFC-8468-D85CE41A0C3C) documentation.


In [ ]:
# Create a KM model object and fit it according to the training data and parameter 
# settings. n_clusters parameter specifies the number of clusters.
km_mod = oml.km(n_clusters = 4, **setting).fit(train_ltv, case_id = 'CUST_ID')

**Note** : To understand this model, visit the [k-Means](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/k-means.html#GUID-7909D96B-D3B9-411B-BAD5-96DAFAF06B42) page in OML user guide.


In [ ]:
# Show KM model details. Did your model converge?
km_mod

In [ ]:
# Use the model to cluster the test data.
predictions = km_mod.predict(test_ltv,
                 supplemental_cols = test_ltv[:, ['CUST_ID', 'LTV']])
predictions

In [ ]:
# Left merge the oml_cust full data set to view if there is a connection 
# between the four clusters and the predefined classes stored in LTV_BIN column.
km_mod.predict(test_ltv,
               supplemental_cols = test_ltv[:, 
                       ['CUST_ID']]).merge(other=oml_cust[['CUST_ID',
                                                     'LTV_BIN']], on="CUST_ID")

In [ ]:
# Build a summary to count cluster members grouped by life-time value.
km_mod.predict(test_ltv,
   supplemental_cols = test_ltv[:, 
         ['CUST_ID']]).merge(other=oml_cust[['CUST_ID',
                        'LTV_BIN']], on="CUST_ID").crosstab('LTV_BIN_r', 
                                 'CLUSTER_ID_l').sort_values(by = ['CLUSTER_ID_l','count'])

In [ ]:
# What is the relationship between the four clusters discovered by k-Means 
# clustering algorithm and the long-term values?
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,10)
plt.plot(predictions[predictions['CLUSTER_ID'] == 3].pull()[['CLUSTER_ID']].replace({3:1}), 
         predictions[predictions['CLUSTER_ID'] == 3].pull()[['LTV']], 'ro',
         predictions[predictions['CLUSTER_ID'] == 5].pull()[['CLUSTER_ID']].replace({5:2}), 
         predictions[predictions['CLUSTER_ID'] == 5].pull()[['LTV']], 'go',
         predictions[predictions['CLUSTER_ID'] == 6].pull()[['CLUSTER_ID']].replace({6:3}), 
         predictions[predictions['CLUSTER_ID'] == 6].pull()[['LTV']], 'bo',
         predictions[predictions['CLUSTER_ID'] == 7].pull()[['CLUSTER_ID']].replace({7:4}), 
         predictions[predictions['CLUSTER_ID'] == 7].pull()[['LTV']], 'mo', alpha=0.15)
plt.xlabel('Cluster 3-red, 5-green, 6-blue, 7-magenta')
plt.ylabel('LTV')
plt.show()

In [ ]:
# Calculate the score value based on the test data.
km_mod.score(test_ltv)

## **Task 6:** Rank customer attributes with Attribute Importance

Oracle Machine Learning supports the Attribute Importance (AI) machine learning function, which ranks attributes according to their importance. Attribute importance does not actually select the features, but ranks them as to their relevance to predicting the result. It is up to the user to review the ranked features and create a data set to include the desired features. 

> **Note** : For more information, visit [About Feature Selection and Attribute Importance](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/feature-selection.html#GUID-FE2DFE18-670E-4E1A-83A8-5C67CA4D8564) documentation.

In this example, you will use the entire data set, however you will drop `LTV` column. This column represents the exact numeric value of your customers LTV, and you already know there is a direct relationship between this column and the customer class represented in `LTV_BIN` column, so LTV value cannot be considered an attribute. 


In [ ]:
# Create an OML data frame proxy object in Python that represents your Oracle Database data set.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust = oml_cust.drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set into training and test data. Default 70% train, 30% test. 
# train_x are the customer features, and train_y specifies the label for each customer, 
# in this case LTV_BIN value.
dat = oml_cust.split() 
train_x = dat[0].drop('LTV_BIN') 
train_y = dat[0]['LTV_BIN']
test_dat = dat[1]

In [ ]:
# Verify features data frame shape, number or rows and columns.
train_x.shape

In [ ]:
# Verify test data frame shape. Why are they different?
test_dat.shape

In [ ]:
# Specify model settings.
setting = {'ODMS_SAMPLING':'ODMS_SAMPLING_DISABLE'}

**Note** : For the complete list of settings, check the [`DBMS_DATA_MINING` — Global Settings](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-24047A09-0542-4870-91D8-329F28B0ED75) table.


In [ ]:
# Create an AI model object.
ai_mod = oml.ai(**setting)

**Note** : To understand this model, visit the [Attribute Importance](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/attribute-importance.html#GUID-01F5EB3D-68C2-4EF4-A1BB-AF4320A0246A) page in OML user guide.


In [ ]:
# Fit the AI model according to the training data and parameter settings.
ai_mod = ai_mod.fit(train_x, train_y)

In [ ]:
# Show the model details.
ai_mod

What rank does it specify for the marital status feature? Is this feature important enough as you considered in the previous step?

## **Task 7:** Perform feature extraction using Singular Value Decomposition

Singular Value Decomposition (SVD) is an unsupervised algorithm for feature extraction. SVD is an orthogonal linear transformation that is optimal at capturing the underlying variance of the data. This property is very useful for reducing the dimensionality of high-dimensional data and for supporting meaningful data visualizations.

> **Note** : For more information, visit [Singular Value Decomposition](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/singular-value-decomposition.html#GUID-703B237F-D9C5-4543-97DD-31A914BB6A05) documentation.

At this step, you want to identify the most important features and how are these related to your customers attributes.


In [ ]:
# Create an OML data frame proxy object in Python that represents your Oracle Database data set.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust = oml_cust.drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set and create training and test data.
ltv_dat = oml_cust.split() 
train_ltv = ltv_dat[0]
test_ltv = ltv_dat[1]

In [ ]:
# Specify settings.
setting = {'SVDS_SCORING_MODE':'SVDS_SCORING_PCA', 'ODMS_DETAILS':'ODMS_ENABLE'}

**Note** : For the complete list of settings, check the [`DBMS_DATA_MINING` — Global Settings](https://docs.oracle.com/en/database/oracle/oracle-database/21/arpls/DBMS_DATA_MINING.html#GUID-24047A09-0542-4870-91D8-329F28B0ED75) table.


In [ ]:
# Create an SVD model object.
svd_mod = oml.svd(**setting)

**Note** : To understand this model, visit the [Singular Value Decomposition](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/singular-value-decomposition.html#GUID-FA08B2D3-3DF4-4311-A30F-0DB085C7AFED) page in OML user guide.


In [ ]:
# Fit the model according to the training data and parameter settings showing the model details.
svd_mod = svd_mod.fit(train_ltv, case_id = 'CUST_ID')
svd_mod

In [ ]:
# Use the model to make predictions on the test data.
predictions = svd_mod.predict(test_ltv,
                 supplemental_cols = test_ltv[:, ['CUST_ID','LAST',
                                                  'FIRST','LTV_BIN']])
predictions

In [ ]:
# List the most important features for customers classified as VERY HIGH.
predictions[predictions['LTV_BIN'] 
        == 'VERY HIGH'].crosstab('LTV_BIN',
                                 'FEATURE_ID').sort_values('count', 
                                                               ascending=False)

In [ ]:
# What about the most important features for customers classified as HIGH?
predictions[predictions['LTV_BIN'] 
        == 'HIGH'].crosstab('LTV_BIN',
                                 'FEATURE_ID').sort_values('count', 
                                                               ascending=False)

In [ ]:
# Verify the most important features for customers classified as MEDIUM.
predictions[predictions['LTV_BIN'] 
        == 'MEDIUM'].crosstab('LTV_BIN',
                                 'FEATURE_ID').sort_values('count', 
                                                               ascending=False)

In [ ]:
# Finally, list the most important features for customers classified as LOW. Draw a conclusion.
predictions[predictions['LTV_BIN'] 
        == 'LOW'].crosstab('LTV_BIN',
                                 'FEATURE_ID').sort_values('count', 
                                                               ascending=False)

In [ ]:
# Perform dimensionality reduction and return values for the two features 
# that have the highest topN values.
svd_mod.transform(test_ltv,
                  supplemental_cols = test_ltv[:, ['CUST_ID', 'LTV_BIN']], 
                         topN = 2).sort_values(by = ['CUST_ID', 'TOP_1', 'TOP_1_VAL'])

In [ ]:
# List the most important customer attributes for the most important feature.
svd_mod.features[svd_mod.features['FEATURE_ID'] 
                 == 1].sort_values('VALUE', ascending=False).head()

## **Task 8:** Automated Machine Learning: Algorithm Selection

Automated Machine Learning (AutoML) provides a built-in expert system for data analytics and modeling that you can employ to build machine learning models.

Any modeling problem for a specified data set and prediction task involves a sequence of data cleansing and preprocessing, algorithm selection, and model tuning tasks. Each of these steps require data science expertise to help guide the process to an efficient final model. Automated Machine Learning (AutoML) automates this process with its built-in data science expertise.

> **Note** : For more information, visit [About Association](https://docs.oracle.com/en/database/oracle/machine-learning/oml4sql/21/dmcon/association.html#GUID-2FE196F3-94C5-4EDB-9AEC-40DCB43E8A89) documentation.

At this step, you will use AutoML to select the best Oracle Machine Learning algorithm based on the characteristics of the data set and the task. No single algorithm works best for all modeling problems. AutoML ranks the candidate algorithms according to how likely each is to produce a quality model. Compare this to the first part of the workshop where you experimented with Decision Tree and Neural Networks algorithms manually.


In [ ]:
# Import automl from oml Python library.
from oml import automl

In [ ]:
# Create an OML data frame proxy object that represents the database table. 
# Create two data sets, one for classification task called oml_cust_c, and 
# another for regression task called oml_cust_r.
oml_cust = oml.sync(table = "CUST_INSUR_LTV")
oml_cust_c = oml_cust.drop('LTV')
oml_cust_r = oml_cust.drop('LTV_BIN')

In [ ]:
# Split the data set into training and test data for classification task.
train, test = oml_cust_c.split(ratio=(0.8, 0.2), seed = 1234) 
X, y = train.drop('LTV_BIN'), train['LTV_BIN']
X_test, y_test = test.drop('LTV_BIN'), test['LTV_BIN']

In [ ]:
# Create an automated algorithm selection object with f1_macro as the score_metric argument.
asel_c = automl.AlgorithmSelection(mining_function='classification',
                                   score_metric='f1_macro', parallel=4)

**Note** : For more information, visit [Classification and Regression Metrics](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/about-automl.html#GUID-9F514C2B-1772-4073-807F-3E829D5D558C) documentation.


In [ ]:
# Run algorithm selection to get the top k predicted algorithms and their ranking without tuning.
algo_ranking_c = asel_c.select(X, y, k=3)

In [ ]:
# Show the selected and tuned model.
[(m, "{:.2f}".format(s)) for m,s in algo_ranking_c]

In [ ]:
# Split the data set into training and test data for regression task.
train, test = oml_cust_r.split(ratio=(0.8, 0.2), seed = 1234) 
X, y = train.drop('LTV'), train['LTV']
X_test, y_test = test.drop('LTV'), test['LTV']

In [ ]:
# Create an automated algorithm selection object with f1_macro as the score_metric argument.
asel_r = automl.AlgorithmSelection(mining_function='regression',
                                   score_metric='r2', parallel=4)

In [ ]:
# Run algorithm selection to get the top k predicted algorithms and their ranking without tuning.
algo_ranking_r = asel_r.select(X, y, k=3)

In [ ]:
# Show the selected and tuned model.
[(m, "{:.2f}".format(s)) for m,s in algo_ranking_r]

As you can see, AutoML is able to provide you the algorithms and a ranking of the algorithms best for the data set automatically for the Classification and Regression machine learning types.

## **Task 9:** Automated Machine Learning: Feature Selection

AutoML Feature Selection identifies the most relevant feature subsets for a training data set and an Oracle Machine Learning algorithm. In a data analytics application, feature selection is a critical data preprocessing step that has a high impact on both runtime and model performance.

> **Note** : For more information on this algorithm, visit [Feature Selection](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/feature-selection.html#GUID-576E9C37-6743-4DCC-8939-44772A5C41AB) documentation.

In this AutoML example, you will reduce the number of features of your customers data set, and compare the accuracy score of a Support Vector Machine (SVM) algorithm for a classification task.


In [ ]:
# Create an OML data frame proxy object in Python that represents your Oracle Database data set.
oml_cust = oml.sync(table = "CUST_INSUR_LTV").drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set into training and test data for classification task.
train, test = oml_cust.split(ratio=(0.8, 0.2), seed = 1234,
                             strata_cols = 'LTV_BIN') 
X_train, y_train = train.drop('LTV_BIN'), train['LTV_BIN']
X_test, y_test = test.drop('LTV_BIN'), test['LTV_BIN']

In [ ]:
# Default SVM model performance before feature selection.
mod = oml.svm(mining_function='classification').fit(X_train, y_train)
"{:.2}".format(mod.score(X_test, y_test))

In [ ]:
# Create an automated feature selection object with accuracy as the score_metric.
fs = automl.FeatureSelection(mining_function='classification',
                             score_metric='accuracy', parallel=4)

In [ ]:
# Get the reduced feature subset on the train data set. How many features 
# have been reduced from the original data set?
subset = fs.reduce('svm_linear', X_train, y_train)
"{} features reduced to {}".format(len(X_train.columns),
                                   len(subset))

In [ ]:
# Use the subset to select the features and create a SVM model on the new 
# reduced data set. Did your model accuracy improve with less features?
X_new = X_train[:,subset]
X_test_new = X_test[:,subset]
mod = oml.svm(mining_function='classification').fit(X_new, y_train) 
"{:.2} with {:.1f}x feature reduction".format(mod.score(X_test_new, y_test), 
                                              len(X_train.columns)/len(X_new.columns))

In [ ]:
# For reproducible results, add CUST_ID column unique case identifier.
train, test = oml_cust.split(ratio=(0.8, 0.2), seed = 1234,
                             hash_cols='CUST_ID', strata_cols = 'LTV_BIN') 
X_train, y_train = train.drop('LTV_BIN'), train['LTV_BIN']
X_test, y_test = test.drop('LTV_BIN'), test['LTV_BIN']

In [ ]:
# Provide the CUST_ID column name to the feature selection reduce function. 
# Does it reduce more the number of features when case ID is provided?
subset = fs.reduce('svm_linear', X_train,
                   y_train, case_id='CUST_ID') 
"{} features reduced to {} with case_id".format(len(X_train.columns)-1, len(subset))

## **Task 10:** Automated Machine Learning: Model Selection

AutoML Model Selection automatically selects an Oracle Machine Learning algorithm according to the selected score metric and then tunes that algorithm. It supports classification and regression algorithms.

> **Note** : For more information, visit [Model Selection](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/model-selection.html#GUID-E2C3D9D2-D685-4AF7-8A04-32127A5CCF07) documentation.

In this example, you will create an AutoML `ModelSelection` object and then use this object to select and tune the best model for your task.


In [ ]:
# Create an OML data frame proxy object that represents your database table.
oml_cust = oml.sync(table = "CUST_INSUR_LTV").drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set into training and test data for classification.
train, test = oml_cust.split(ratio=(0.8, 0.2), seed = 1234) 
X, y = train.drop('LTV_BIN'), train['LTV_BIN']
X_test, y_test = test.drop('LTV_BIN'), test['LTV_BIN']

In [ ]:
# Create an automated model selection object with f1_macro as the score_metric argument.
ms = automl.ModelSelection(mining_function='classification',
                           score_metric='f1_macro', parallel=4)

In [ ]:
# Run model selection to get the top (k=1) predicted algorithm, that defaults to the tuned model.
select_model = ms.select(X, y, k=1)

In [ ]:
# Show the selected and tuned model.
select_model

## **Task 11:** Automated Machine Learning: Model Tuning

AutoML Model Tuning tunes the hyperparameters for the specified classification or regression algorithm and training data. This feature automates the tuning process using a highly-parallel, asynchronous gradient-based hyperparameter optimization algorithm. 

> **Note** : For more information on this algorithm, visit [Model Tuning](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/model-tuning.html#GUID-0F5153CC-39E5-4189-9615-09D8F39D7FBF) documentation.

In this case, you will execute an automated model tuning process for a classification task using Decision Tree (DT) algorithm.


In [ ]:
# Create an OML data frame proxy object in Python that represents your Oracle Database data set.
oml_cust = oml.sync(table = "CUST_INSUR_LTV").drop('LTV')
oml_cust.head()

In [ ]:
# Split the data set into training and test data for classification.
train, test = oml_cust.split(ratio=(0.8, 0.2), seed = 1234) 
X, y = train.drop('LTV_BIN'), train['LTV_BIN']
X_test, y_test = test.drop('LTV_BIN'), test['LTV_BIN']

In [ ]:
# Start an automated model tuning run with a DT model.
at = automl.ModelTuning(mining_function='classification', parallel=4, 
                        score_metric='accuracy')
results = at.tune('dt', X, y)

**Note** : For more information, visit [Classification and Regression Metrics](https://docs.oracle.com/en/database/oracle/machine-learning/oml4py/1/mlpug/about-automl.html#GUID-9F514C2B-1772-4073-807F-3E829D5D558C) documentation.


In [ ]:
# Show the DT tuned model details.
tuned_model = results['best_model'] 
tuned_model

In [ ]:
# Show the best tuned model train score and the corresponding hyperparameters.
score, params = results['all_evals'][0] 
"{:.2}".format(score), ["{}:{}".format(k, params[k])
                        for k in sorted(params)]

In [ ]:
# Use the DT tuned model to get the score on the test set. How is the 
# score on the test data compared to the score on training data?
"{:.2}".format(tuned_model.score(X_test, y_test))

## Acknowledgements
* **Authors** - Milton Wan, Valentin Leonard Tabacaru
* **Last Updated By/Date** -  Valentin Leonard Tabacaru, July 2021
    
## Need Help?
Please submit feedback or ask for help using our [LiveLabs Support Forum](https://community.oracle.com/tech/developers/categories/livelabsdiscussions). Please click the **Log In** button and login using your Oracle Account. Click the **Ask A Question** button to the left to start a *New Discussion* or *Ask a Question*.  Please include your workshop name and lab name.  You can also include screenshots and attach files.  Engage directly with the author of the workshop.
    
If you do not have an Oracle Account, click [here](https://profile.oracle.com/myprofile/account/create-account.jspx) to create one.